<a href="https://colab.research.google.com/github/nikita-0209/image-classification/blob/master/CNN_Architecture%3A%20Default%20CIFAR%20Image%20Size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [0]:
from tensorflow.python.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.python.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import numpy as np

import os

import pandas as pd

import random

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Load Dataset

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1J615QNajDV3Q89TnE8R-iIvj_5zhMo1J"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_images.npy')        # replace the file name with your file

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1tCHOmcsg96D9xfwY1Lf2-xZPT1PEUePe"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_labels.csv')        # replace the file name with your file

In [0]:
#Dataset
downloaded = drive.CreateFile({'id':"1ZdU78f7JN-OUI3Gd0UCFMwbr9RnoHk6G"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_images.npy')        # replace the file name with your file

## Preprocessing

In [0]:
testX=np.load('test_images.npy')

In [0]:
trainX=np.load('train_images.npy')

In [0]:
trainY=pd.read_csv('train_labels.csv')

In [0]:
validationX=trainX[40000:, :]
trainX=trainX[:40000, :]

In [0]:
validationY=trainY.iloc[40000:, 1]
trainY=trainY.iloc[:40000, 1]

In [0]:
def preprocessX(data):
  size=data.shape[0]
  data_new=np.zeros((size, 32, 32,3 ))
  for i in range (size):
    data_new[i]=np.dstack((data[i][0], data[i][1], data[i][2]))
    i=i+1
  
  return data_new

In [0]:
def preprocessY (lst):
  integer_encoded = label_encoder.fit_transform(np.array(lst))
  trainY=to_categorical(integer_encoded)
  return trainY

In [0]:
trainY=preprocessY(trainY)
validationY=preprocessY(validationY)

In [0]:
trainX=preprocessX(trainX)
validationX=preprocessX(validationX)

In [0]:
input_shape=(32, 32, 3)
batch_size=64

## Model

In [0]:
#Define Model
model=Sequential()
model.add(Conv2D(128, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss=CategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=['accuracy'])


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 128)       9728      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 128)       409728    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               819456    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

## Image Generator

In [0]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08) 

## Checkpoints

In [0]:
# Include the epoch in the file name (uses `str.format`)

checkpoint_path = '/content/drive/My Drive/ML Projects/CIFAR/Checkpoints_CNN/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=1)


## Reload Weights
### Whenever Colab crashed, weights were reloaded before resuming training.

In [0]:
model.load_weights('/content/drive/My Drive/ML Projects/CIFAR/Checkpoints_CNN/cp-0024.ckpt')

## Training Model
### After reloading the previous 24 epochs, we resume training.
### Training interrupted as accuracy had become roughly constant


In [0]:
history=model.fit_generator(gen.flow(trainX, trainY, batch_size), 
                    steps_per_epoch=trainY.shape[0]//batch_size, 
                    epochs=25, 
                    validation_data=gen.flow(validationX, validationY, batch_size),
                    validation_steps=validationY.shape[0]//batch_size,
                    callbacks=[cp_callback]
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
625/625 [==============================] - ETA: 0s - loss: 0.9292 - accuracy: 0.6813
Epoch 00001: saving model to /content/drive/My Drive/ML Projects/CIFAR/Checkpoints_CNN/cp-0001.ckpt
625/625 [==============================] - 342s 547ms/step - loss: 0.9292 - accuracy: 0.6813 - val_loss: 1.2454 - val_accuracy: 0.6117
Epoch 2/25
625/625 [==============================] - ETA: 0s - loss: 0.9313 - accuracy: 0.6821
Epoch 00002: saving model to /content/drive/My Drive/ML Projects/CIFAR/Checkpoints_CNN/cp-0002.ckpt
625/625 [==============================] - 340s 543ms/step - loss: 0.9313 - accuracy: 0.6821 - val_loss: 1.2116 - val_accuracy: 0.6081
Epoch 3/25
625/625 [==============================] - ETA: 0s - loss: 0.9173 - accuracy: 0.6861
Epoch 00003: saving model to /content/drive/My Drive/ML Projects/CIFAR/Checkpoints_CNN/cp-0003.ckpt
625/625 [==============================] - 342s 547ms/step - loss:

KeyboardInterrupt: ignored